<img src="http://sct.inf.utfsm.cl/wp-content/uploads/2020/04/logo_di.png" style="width:60%">

<center>
    <h1>ILI285/INF285 Computación Científica </h1>
    <h1>Pauta Pregunta de "Splines" - COP3</h1>
</center>

In [1]:
M=8
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# Enunciado

Sean los pares de puntos $(x_i,y_i)$, para $i$ de 1 a 6. Construya la siguiente Spline Cubica. Aca  $(x_i,y_i)$ podían ser:

In [2]:
'''
En su prueba usted pudo haber recibido uno de las siguientes datasets:
''' 

Datasets = {'Dataset1': (np.array([1,2,4,5,7,9]), np.array([0.2919265817264289, 0.17317818956819406,
                                                        0.4272499830956933, 0.0804642354617738,
                                                        0.5683686091039167, 0.83015835412204])),
            'Dataset2': (np.array([0,1,2,3,4,5]), np.array([0.0, 0.7080734182735712,
                                                            0.826821810431806,0.01991485667481699,
                                                            0.5727500169043067,0.9195357645382262])),
            'Dataset3': (np.array([2,3,4,5,6,7]), np.array([0.7518269446689928, 0.0028103847344612735,
                                                            -0.4334586419808374, -0.881765166036633,
                                                            -0.021814811956275375, 0.28357603940507775]))}

Construya la siguiente Spline Cubica:

$$\begin{equation*}{S(x) =\begin{cases}S_1(x),&\text{si }x\in [{x_1},{x_2}]\\S_2(x),& \text{si } x \in [{x_2},{x_3}]\\S_3(x),&\text{si }x\in [{x_3},{x_4}]\\S_4(x), & \text{si } x \in [{x_4},{x_5}]\\S_5(x),&\text{si }x\in [{x_5},{x_6}]\end{cases}}\end{equation*}$$

donde $$S_i(x) = y_i + a_i (x-x_i) + b_i (x-x_i)^{2} + c_i (x-x_i)^{3}$$ utilizando condiciones de borde del tipo **Robin/Clamped con derivada igual a 2** en $x_1$ y del tipo **Natural/Robin** en $x_6$. Considerar que la condición de tipo Robin se define como:

$$
{\displaystyle {\begin{cases}\alpha\, S_1(x_1)-\beta \, S_1'(x_1)=g\\ \alpha \,S_5(x_6)+\beta \, S_5'(x_6)=g.\,\end{cases}}}
$$

Indique cuales son los 4 números después de la coma (sin redondear) del coeficiente $b_4$. Donde según la prueba los valores de $\alpha$ toma los valores $\{4,5,6\}$, $\beta$ toma los valores $\{5,6,7\}$ y $g$ toma los valores $\{1,2,3\}$.

Como las condiciones de tipo Clamped y Natural ya han sido implementadas en los jupyter notebook del curso nos enfocaremos en estudiar como implementar las condiciones de tipo Robin en ambos extremos del dominio. Comenzaremos encontrando la primera derivada de $S_1(x)$ y de $S_2(x)$.

$$
S_1(x) = y_1 + a_1 (x-x_1) + b_1 (x-x_1)^{2} + c_1 (x-x_1)^{3} \\
S'_1(x) = a_1 + 2 b_1 (x-x_1) + 3 c_1 (x-x_1)^{2}\\
S_5(x) = y_5 + a_5 (x-x_5) + b_5 (x-x_5)^{2} + c_5 (x-x_5)^{3} \\
S'_5(x) = a_5 + 2 b_5 (x-x_5) + 3 c_5 (x-x_5)^{2}\\
$$

Si hubiese tocado la condición de tipo robin en $x_1$ entonces la condición de borde queda:

$$
\alpha (y_1 + a_1 (x-x_1) + b_1 (x-x_1)^{2} + c_1 (x-x_1)^{3}) - \beta(a_1 + 2 b_1 (x-x_1) + 3 c_1 (x-x_1)^{2}) = g 
$$

Ordenando la ecuación anterior nos queda:

$$
(\alpha (x-x_1) - \beta)a_1+  (\alpha (x-x_1)^{2} - 2 \beta (x-x_1))b_1 + ((x-x_1)^{3} - 3 \beta (x-x_1)^{2}) c_1 = g - \alpha y_1
$$

Luego evaluando en $x_1$ tenemos:

$$
(\alpha (x_1-x_1) - \beta)a_1+  (\alpha (x_1-x_1)^{2} - 2 \beta (x_1-x_1))b_1 + ((x_1-x_1)^{3} - 3 \beta (x_1-x_1)^{2}) c_1 = g - \alpha y_1 \\
- \beta a_1 = g - \alpha y_1
$$

Si hubiese tocado la condición del tipo Robin en $x_6$ entonces la condición de borde queda:

$$
\alpha (y_5 + a_5 (x-x_5) + b_5 (x-x_5)^{2} + c_5 (x-x_5)^{3}) + \beta(a_5 + 2 b_5 (x-x_5) + 3 c_5 (x-x_5)^{2}) = g 
$$

Ordenando la ecuación anterior nos queda:

$$
(\alpha (x-x_5) + \beta)a_5+  (\alpha (x-x_5)^{2} + 2 \beta (x-x_5))b_5 + ((x-x_5)^{3} - 3 \beta (x-x_5)^{2}) c_5 = g - \alpha y_5
$$

Luego evaluando en $x_6$ tenemos:

$$
(\alpha (x_6-x_5) + \beta)a_5+  (\alpha (x_6-x_5)^{2} + 2 \beta (x_6-x_5))b_5 + ((x_6-x_5)^{3} - 3 \beta (x_6-x_5)^{2}) c_5 = g - \alpha y_5
$$

La siguiente implementación esta basada en el código del profesor Claudio Torres. Se modifica unicamente las ultimas dos filas de la matriz $A$ y los ultimos dos elementos del vector $b$ ya que estos representan las dos ecuaciones que se deben imponer para construir una spline con las condiciones de borde requerida. 

In [3]:
def cubic_spline(x, y, start = 'Robin1', end = 'Robin2', p1=2, a_r = 1, b_r = 1, g = None):
    
    '''
    x: x-coordinates of points
    y: y-coordinates of points
    start: Border condition at x_0 (Robin, Clamped)
    end: Border condition at x_n (Robin, Natural)
    p1: Clamped condition
    a_r: alpha for robin border condition
    b_r: beta for robin border condition
    g: RHS for robin border condition
    '''
    
    if g == None:
        g = lambda x: 0
    
    n = len(x)
    A = np.zeros((3*n-3, 3*n-3))
    b = np.zeros(3*n-3)
    
    delta_x=np.diff(x)
       
    #Building the linear system of equations
    
    #1st property
    for i in np.arange(n-1):
        b[i]= y[i+1]-y[i]
        A[i,3*i:3*(i+1)] = [delta_x[i],delta_x[i]**2,delta_x[i]**3]
    #2nd property
    for i in np.arange(n-2):
        A[(n-1)+i,3*i:3*(i+1)+1]=[1, 2*delta_x[i], 3*delta_x[i]**2, -1]
    #3rd property
    for i in np.arange(n-2):
        A[(n-1)+(n-2)+i,3*i:3*(i+1)+2] = [0, 2, 6*delta_x[i], 0, -2]
    
    #Ending conditions (4th property)
    
    #Robin condition at x_0
    if start =='Robin1':
        A[-2,0]= -b_r
        b[-2] = g - a_r*y[0]
        
    #Clamped condition at x_0
    elif start == 'Clamped1':
        A[-2,0] = 1
        b[-2] = p1
        
    #Robin condition at x_n
    if end == 'Robin2':
        A[-1,-1] = a_r*delta_x[-1]**3 + b_r*3*delta_x[-1]**2
        A[-1,-2] = a_r*delta_x[-1]**2 + b_r*2*delta_x[-1]
        A[-1,-3] = a_r*delta_x[-1] + b_r
        b[-1] = g - a_r*y[-1]
        
    #Robin condition at x_n
    elif end == 'Natural2':
        A[-1,-2] = 2
        A[-1,-1] = 6*delta_x[-1]
        
    sol = np.linalg.solve(A,b)
    #Note that the question asked for the coefficient that accompanies the square term,
    #in this case 'c'
    S = {'b':sol[::3],
         'c':sol[1::3],
         'd':sol[2::3],
         'x':x,
         'y':y
        }
    return S

In [4]:
def find_coeficient(dataset, border, alpha, beta, g):
    border_condition = {'Robin-Natural': ['Robin1', 'Natural2'],
                            'Clamped-Robin': ['Clamped1','Robin2']}
    x, y = Datasets[dataset]
    S = cubic_spline(x, y, start = border_condition[border][0], 
                     end = border_condition[border][1], 
                     a_r = alpha, b_r = beta, g = g)
    print(r"x:",x)
    print(r"y:",y)
    print("b:", str(S['c'][3]).split(".")[1][:4])
    
interact(find_coeficient, dataset=['Dataset1','Dataset2','Dataset3'],
         border = ['Robin-Natural','Clamped-Robin'],alpha = [4,5,6], 
         beta = [5,6,7], g = [1,2,3] );

interactive(children=(Dropdown(description='dataset', options=('Dataset1', 'Dataset2', 'Dataset3'), value='Dat…